# Navier Stokes Equation Reduced Basis Method 
Incompressible timedependet Navier Stokes equation

Find $(u,p):[0,T] \to (H_{0,D}^1)^d \times L^2$, s.t.

\begin{align}
\int_{\Omega} \partial_t u \cdot v + \int_{\Omega} \nu \nabla u \nabla v + u \cdot \nabla u v - \int_{\Omega} \operatorname{div}(v) p &= \int f v  && \forall v \in (H_{0,D}^1)^d, \\ 
- \int_{\Omega} \operatorname{div}(u) q &= 0 && \forall q \in L^2, \\
\quad u(t=0) & = u_0
\end{align}

In [1]:
from netgen import gui
from ngsolve import *
from netgen.geom2d import SplineGeometry
import scipy.sparse as sp
import numpy as np
from math import pi

In [2]:
from netgen.geom2d import SplineGeometry
geo = SplineGeometry()
geo.AddRectangle((0, 0), (2, 0.41), bcs = ("wall", "outlet", "wall", "inlet"))
geo.AddCircle((0.2, 0.2), r=0.05, leftdomain = 0, rightdomain = 1, bc = "cyl")
mesh = Mesh( geo.GenerateMesh(maxh = 0.08) ); Draw(mesh)
order = 3
mesh.Curve(order)
# viscosity
nu = 0.001
dt = 0.001

T = 1
dn = int(T/dt)
z = 100  # Number of snapshots

In [3]:
V1 = HDiv ( mesh, order = order, dirichlet = "wall|cyl|inlet" )
V2 = FESpace ( "vectorfacet", mesh, order = order,
              dirichlet = "wall|cyl|inlet" )
Q = L2( mesh, order = order-1)
V = FESpace ([V1,V2,Q])

u, uhat, p = V.TrialFunction()
v, vhat, q = V.TestFunction()

In [4]:
nu = 0.001 # viscosity
n = specialcf.normal(mesh.dim)
h = specialcf.mesh_size
def tang(vec):
    return vec - (vec*n)*n

In [5]:
alpha = 4  # SIP parameter
dS = dx(element_boundary=True)
a = BilinearForm ( V, symmetric=True)
a += nu * InnerProduct ( Grad(u), Grad(v)) * dx
a += nu * InnerProduct ( Grad(u)*n,  tang(vhat-v) ) * dS
a += nu * InnerProduct ( Grad(v)*n,  tang(uhat-u) ) * dS
a += nu * alpha*order*order/h * InnerProduct(tang(vhat-v),  tang(uhat-u))*dS
a += (-div(u)*q -div(v)*p) * dx
a.Assemble()

rows,cols,vals = a.mat.COO()
Ah = sp.csr_matrix((vals,(rows,cols)))

In [6]:
m = BilinearForm(V , symmetric=True)
m += SymbolicBFI( u * v )
m.Assemble()

In [7]:
gfu = GridFunction(V)

U0 = 1.5
uin = CoefficientFunction( (U0*4*y*(0.41-y)/(0.41*0.41),0) )
gfu.components[0].Set(uin, definedon=mesh.Boundaries("inlet"))

invstokes = a.mat.Inverse(V.FreeDofs(), inverse="umfpack")
gfu.vec.data += invstokes @ -a.mat * gfu.vec

Draw( gfu.components[0], mesh, "velocity" )
Draw( gfu.components[2], mesh, "pressure" )
Draw( Norm(gfu.components[0]), mesh, "absvel(hdiv)")

In [8]:
VL2 = VectorL2(mesh, order=order, piola=True)
uL2, vL2 = VL2.TnT()
bfmixed = BilinearForm ( V, VL2, nonassemble=True )
bfmixed += vL2*u*dx
bfmixedT = BilinearForm ( VL2, V, nonassemble=True)
bfmixedT += uL2*v*dx



In [9]:
vel = gfu.components[0]
convL2 = BilinearForm(VL2, nonassemble=True )
convL2 += (-InnerProduct(Grad(vL2) * vel, uL2)) * dx
un = InnerProduct(vel,n)
upwindL2 = IfPos(un, un*uL2, un*uL2.Other(bnd=uin))

dskel_inner  = dx(skeleton=True)
dskel_bound  = ds(skeleton=True)

convL2 += InnerProduct (upwindL2, vL2-vL2.Other()) * dskel_inner
convL2 += InnerProduct (upwindL2, vL2) * dskel_bound



In [10]:
class PropagateConvection(BaseMatrix):
    def __init__(self,tau,steps):
        super(PropagateConvection, self).__init__()
        self.tau = tau; self.steps = steps
        self.h = V.ndof; self.w = V.ndof # operator domain and range
        self.mL2 = VL2.Mass(Id(mesh.dim)); self.invmL2 = self.mL2.Inverse()
        self.vecL2 = bfmixed.mat.CreateColVector() # temp vector
    def Mult(self, x, y):
        self.vecL2.data = self.invmL2 @ bfmixed.mat * x # <- project from Hdiv to L2
        for i in range(self.steps):
            self.vecL2.data -= self.tau/self.steps * self.invmL2 @ convL2.mat * self.vecL2
        y.data = bfmixedT.mat * self.vecL2
    def CreateColVector(self):
        return CreateVVector(self.h)



In [11]:
t = 0; tend = 0
tau = 0.01; substeps = 10

mstar = m.mat.CreateMatrix()
mstar.AsVector().data = m.mat.AsVector() + tau * a.mat.AsVector()
inv = mstar.Inverse(V.FreeDofs(), inverse="umfpack")



In [12]:
erg_h = np.zeros([len(gfu.vec),z])
V_GM = np.zeros([len(gfu.vec),z])

V_sb = MultiVector(gfu.vec, 1)
V_sb[0] = gfu.vec
V_GM[:,0] = np.array(V_sb[0])/np.linalg.norm(np.array(V_sb[0]))

In [13]:
%%time
tend += 1
o = 1
res = gfu.vec.CreateVector()
convpropagator = PropagateConvection(tau,substeps)
while t < tend:
    gfu.vec.data += inv @ (convpropagator - mstar) * gfu.vec
    t += tau
    with TaskManager():
        V_sb.AppendOrthogonalize(gfu.vec)
    if o < z:
        V_GM[:,o] = np.array(V_sb[o])
    o = o + 1
    #print ("\r  t =", t, end="")
    Redraw(blocking=True)

CPU times: user 14.8 s, sys: 772 ms, total: 15.6 s
Wall time: 9.99 s


In [14]:
print(V_GM[:,0].dot(V_GM[:,1]))

-9.922618282587337e-16


In [15]:
AN = np.transpose(V_GM).dot(Ah.dot(V_GM))

In [16]:
AN.shape

(100, 100)